In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import time

# Load your dataset
filename = "NF-BOT-IOT"
train_data = pd.read_csv('/content/drive/MyDrive/dataset/NF-BOT-IOT_train_preprocessed.csv', sep=',', encoding='utf-8')
test_data = pd.read_csv('/content/drive/MyDrive/dataset/NF-BOT-IOT_test_preprocessed.csv', sep=',', encoding='utf-8')

# Prepare training data
X_train = train_data.drop(columns=['label'], axis=1)
y_train = train_data['label']

# Prepare test data
X_test = test_data.drop(columns=['label'], axis=1)
y_test = test_data['label']

# Split a small portion of training and test data for optimization (ALO)
X_t, _, y_t, _ = train_test_split(X_train, y_train, train_size=0.01, random_state=7)
X_test_t, _, y_test_t, _ = train_test_split(X_test, y_test, train_size=0.01, random_state=7)

# Define custom Ant Lion Optimizer (ALO) algorithm for feature selection
class ALOFeatureSelection:
    def __init__(self, num_lions=5, max_iter=10):
        self.num_lions = num_lions
        self.max_iter = max_iter

    def fit(self, X, y):
        start_time = time.time()  # Start timer
        num_features = X.shape[1]
        x_solutions = np.random.randint(0, 2, size=(self.num_lions, num_features))  # Initialize with binary values
        best_feature_set = None
        best_accuracy = 0.0

        for iteration in range(self.max_iter):
            fitness_values = []

            for solution in x_solutions:
                selected_features = np.where(solution > 0)[0]  # Select features where value > 0

                # Evaluate subset using a simple model (e.g., Random Forest)
                clf = RandomForestClassifier()
                X_subset = X.iloc[:, selected_features]
                clf.fit(X_subset, y)
                accuracy = accuracy_score(y, clf.predict(X_subset))

                fitness_values.append((selected_features, accuracy))

                # Update best solution found
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_feature_set = selected_features

            # Update solutions based on fitness values
            sorted_indices = np.argsort([fitness[1] for fitness in fitness_values])[::-1]
            x_solutions = np.array([x_solutions[idx] for idx in sorted_indices])

        end_time = time.time()  # End timer
        execution_time = end_time - start_time  # Calculate execution time

        self.selected_features = best_feature_set
        self.execution_time = execution_time
        self.num_selected_features = len(best_feature_set)
        return self

# Instantiate and run the ALO algorithm for feature selection on training data
alo = ALOFeatureSelection(num_lions=5, max_iter=10)
alo.fit(X_t, y_t)

# Apply selected features to training data
selected_features_train = X_train.columns[alo.selected_features]

# Create a DataFrame for optimization results
optimization_results = pd.DataFrame({
    "Optimization": ["ALO"],
    "Execution Time of Optimizer": [alo.execution_time],
    "No of Feature Selected": [alo.num_selected_features],
    "Selected Feature": [', '.join(selected_features_train.tolist())]
})

# Save optimization results to a CSV file
optimization_results.to_csv('optimization_results.csv', index=False)
